In [130]:
from utils import *
from functools import reduce, cache, wraps
import itertools
import re
s="""%mh -> rz
%nd -> jx
%xt -> cx
%dp -> mh
%pz -> zg, bf
%rp -> jb, bf
%jb -> bf, kp
%rj -> xt, cx
%hg -> dl, bf
%pt -> gm, vv
%pf -> xk, qr
%cv -> jp, cx
%zg -> bb
%qn -> gm, bh
%kp -> pz
%kg -> gm, pt
%sl -> rp
%dz -> bf, dc
%hm -> cx, tz
%dc -> fk
%xk -> qr, sf
&kr -> gf
%bq -> qr, mg
%sf -> qr
&cx -> ff, vx, zs
%hr -> fq, gm
%ls -> lf, gm
%mf -> cx, sx
%vq -> gm
%sx -> cx, rj
&gm -> kg, kf, fq, nc, lf
%jx -> qr, zz
%tz -> mf, cx
%jp -> cx, kt
%bb -> hg, bf
%zz -> pf, qr
&qr -> dp, bq, nd, rz, mg, qk, mh
%nc -> gb
%kt -> hm, cx
%mg -> dp
%dl -> bf
&zs -> gf
&bf -> dz, zg, kr, sl, fk, kp, dc
%bh -> vq, gm
&kf -> gf
%fq -> qn
%vl -> vx, cx
&qk -> gf
%fk -> sl
%tj -> nd, qr
%gb -> ls, gm
%lf -> hr
%vx -> cv
%ff -> vl, cx
broadcaster -> kg, dz, ff, bq
%vv -> nc, gm
&gf -> rx
%rz -> tj"""

In [131]:
from dataclasses import dataclass
import copy, math

mems = {}
mods = {}
is_flipflop = {}
for co in s.splitlines():
    name, dests = co.split(' -> ')
    if name != 'broadcaster':         
        if name[0] == '%':
            t = False
        else: t = {}
        name = name[1:]
        mems[name] = t
        is_flipflop[name] = t == False
    mods[name] = dests.split(', ')

for k,dests in mods.items():
    for d in dests:
        if d not in mods: continue
        if not is_flipflop[d]:
            mems[d][k] = 0
from collections import deque

def sim():
    hi, lo = 0, 0
    q = deque([('', 'broadcaster',0)])
    while q:
        sender, name, pulse = q.popleft()
        if pulse: hi+=1
        else: lo +=1
        if name not in mods: continue

        dests = mods[name]

        if name == 'broadcaster':
            q.extend([(name, d, pulse) for d in dests])
        elif is_flipflop[name]:
            if pulse == 0:
                q.extend([(name, d, int(not ms[name])) for d in dests])
                ms[name] = not ms[name]
        else:
            ms[name][sender] = pulse
            p = 0 if all(v==1 for v in ms[name].values()) else 1
            q.extend([(name, d, p) for d in dests])
    return hi, lo

ms = copy.deepcopy(mems)
hi=lo=0

for _ in range(1000):
    h, l = sim()
    hi += h
    lo += l

ans(hi*lo)

739960225

In [132]:
rx_sender = [k for k,v in mods.items() if 'rx' in v][0]
feeders = [k for k,v in mods.items() if rx_sender in v]
ls = {}

ms = copy.deepcopy(mems)
def sim2(i):
    i+=1
    q = deque([('', 'broadcaster',0)])
    while q:
        sender, name, pulse = q.popleft()
        if name == 'rx':
            for feeder in feeders:
                if ms[rx_sender][feeder] == 1:
                    ls[feeder] = i
        if name not in mods: continue

        dests = mods[name]

        if name == 'broadcaster':
            q.extend([(name, d, pulse) for d in dests])
        elif is_flipflop[name]:
            if pulse == 0:
                q.extend([(name, d, int(not ms[name])) for d in dests])
                ms[name] = not ms[name]
        else:
            ms[name][sender] = pulse
            p = 0 if all(v==1 for v in ms[name].values()) else 1
            q.extend([(name, d, p) for d in dests])
        

    return i
i = 0
while len(ls) < len(feeders):
    i = sim2(i)

ans(math.lcm(*ls.values()))

231897990075517